<a href="https://colab.research.google.com/github/lokeshsai-m/MTP/blob/main/model-training%20using%20tranformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import tensorflow as tf
!pip install contractions
import contractions
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import re
import transformers
import warnings

lem = WordNetLemmatizer()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 8.0 MB/s eta 0:00:00


In [2]:
data = pd.read_csv("/content/sample_sarcasm.csv")

In [3]:
def preprocess(x):
    x = x.replace("READ MORE","")
    x = x.encode('ascii','ignore')
    x = x.decode()
    x = x.lower()
    x = contractions.fix(x)
    # x = ' '.join([word for word in x.split() if not word in set(stopwords.words('english'))])
    x =  re.sub('[^a-zA-Z0-9]', ' ', x)
    x = ' '.join(x.split())
    # x = lem.lemmatize(x)
    return x

In [4]:
data['processed_tweets'] = data['processed_tweets'].apply(preprocess)

In [5]:
from sklearn.model_selection import train_test_split
X = list(data['processed_tweets'].values)
y = pd.get_dummies(data['sarcasm']).values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [6]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

X_train = tokenizer(X_train, truncation=True, padding=True,max_length=10)
X_test = tokenizer(X_test, truncation=True, padding=True,max_length = 10)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [7]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(X_train),
    y_train
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(X_test),
    y_test
))

In [8]:
from transformers import TFDistilBertForSequenceClassification


model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)


optimizer = tf.keras.optimizers.SGD(learning_rate=10e-5)
model.compile(optimizer=optimizer, loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])
model.fit(train_dataset.shuffle(42).batch(512),
          epochs=5,
          batch_size=512,
          validation_data=val_dataset.shuffle(42).batch(512))

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Epoch 1/5
1/1 [==============================] - 50s 50s/step - loss: 4.2783 - accuracy: 0.3511 - val_loss: 3.8353 - val_accuracy: 0.6000
Epoch 2/5
1/1 [==============================] - 31s 31s/step - loss: 3.6419 - accuracy: 0.4844 - val_loss: 5.6637 - val_accuracy: 0.4200
Epoch 3/5
1/1 [==============================] - 27s 27s/step - loss: 4.2859 - accuracy: 0.4289 - val_loss: 6.2457 - val_accuracy: 0.4000
Epoch 4/5
1/1 [==============================] - 31s 31s/step - loss: 3.7091 - accuracy: 0.3111 - val_loss: 3.1509 - val_accuracy: 0.4000
Epoch 5/5
1/1 [==============================] - 28s 28s/step - loss: 2.7257 - accuracy: 0.3111 - val_loss: 1.6403 - val_accuracy: 0.4000
